In [13]:
import pandas as pd
import numpy as np
import math
import random
import datetime
from collections import Counter

In [14]:
class Node:
    def __init__(self, num_samples):
        self.gain = None
        self.num_samples = num_samples
        self.predicted_class = None
        self.feature = None
        self.threshold = 0
        self.left = None
        self.right = None

class DecisionTreeClassifiers(object):
    def __init__(self, samp_feat_attr, max_depth=None):
        self.max_depth = max_depth
        self.samp_feat_attr = samp_feat_attr

    def fit(self, X, y):
        self.tree_ = self._grow_tree(X, y)
        return self.tree_
        
    def _grow_tree(self, X, y, depth=0):
        node = Node(
            num_samples=y.size,
        )

        # Split recursively until maximum depth is reached.
        if depth < self.max_depth:
            best_feature, best_split, best_gain, l_x_train, r_x_train, l_y_train, r_y_train, max_class_cnt_idx = self._best_split(self.samp_feat_attr,X,y,depth)
            if best_feature is not None:
                node.feature = best_feature
                node.threshold = best_split
                node.gain = best_gain
                node.left = self._grow_tree(l_x_train, l_y_train, depth + 1)
                node.right = self._grow_tree(r_x_train, r_y_train, depth + 1)
            else:
                if y.size == 1:
                    node.feature = None
                    node.predicted_class = y[y.columns[-1]].iloc[0]
                else:
                    node.feature = None
                    node.predicted_class = max_class_cnt_idx
        return node
    
    def _best_split(self, samp_feat_attr, x_train, y_train, depth):
        best_idx, best_split, best_gain = None, None, 0
        best_l_x_train, best_r_x_train, best_l_y_train, best_r_y_train = None, None, None, None
        
        #If input row count is 1 then no need to find gain
        if y_train.size <= 1:
            return None, None, None, None, None, None, None, None
        
        overall = y_train[y_train.columns[0]].value_counts()
        max_class_cnt = None
        max_class_cnt_idx = None
        #If depth reaches max depth then calculate the max count for the class and
        #return as the predicted class
        if depth == self.max_depth-1:
            for val, cnt in overall.iteritems():
                if max_class_cnt is None:
                    max_class_cnt = cnt
                    max_class_cnt_idx = val 
                elif cnt >= max_class_cnt:
                    max_class_cnt = cnt
                    max_class_cnt_idx = val
            return None, None, None, None, None, None, None, max_class_cnt_idx

        for  i in range(len(samp_feat_attr)):
            m = x_train[samp_feat_attr[i]].agg(np.mean)
            df_less = x_train[samp_feat_attr[i]] < m
            df_greater = x_train[samp_feat_attr[i]] >= m
            less_class_count = y_train[df_less][y_train.columns[0]].value_counts()
            greater_class_count = y_train[df_greater][y_train.columns[0]].value_counts()
            l_x_train, r_x_train = x_train[df_less], x_train[df_greater]
            l_y_train, r_y_train = y_train[df_less], y_train[df_greater]

            #calculate left branch impurity
            l_overall = 0
            leftsum = 0
            for val, cnt in less_class_count.iteritems():
                l_overall += cnt
            for val, cnt in less_class_count.iteritems():
                leftsum += (cnt/l_overall)**2
            l_impurity = 1-leftsum

            #calculate right branch impurity
            r_overall = 0
            rightsum = 0
            for val, cnt in greater_class_count.iteritems():
                r_overall += cnt
            for val, cnt in greater_class_count.iteritems():
                rightsum += (cnt/r_overall)**2 
            r_impurity = 1-rightsum

            #overall gain
            gain = 1 - ((l_overall/(l_overall+r_overall))*l_impurity + (r_overall/(l_overall+r_overall)*r_impurity))
            if(gain > best_gain):
                best_gain = gain
                best_split = m
                best_idx = samp_feat_attr[i]
                best_l_x_train, best_r_x_train = l_x_train, r_x_train
                best_l_y_train, best_r_y_train = l_y_train, r_y_train
        return best_idx, best_split, best_gain, best_l_x_train, best_r_x_train, best_l_y_train, best_r_y_train, None
    
    def predict(self, X):
        predicted_row = []
        for index, row in X.iterrows():
            pred_class = self._predict(row)
            predicted_row.append(pred_class)
        return predicted_row

    def _predict(self, inputs):
        node = self.tree_
        while node:
            if node.feature is None:
                return node.predicted_class
            if inputs[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right

In [31]:

""" 
To check the Random Forest accuracy pass the output of the Random forest along with y_test,
This will check the accuracy of predicted classes of all decision trees to that of ground truth
"""
def RF_Accuracy(predicted_classes, y_test):
    class_col_name = None
    class_col_content = []
    for (columnName, columnData) in y_test.iteritems():
        class_col_name = columnName
        class_col_content = columnData.values
        print("Ground truth of Xtest")
    print(class_col_content)
    accuracy_cnt = 0
    
    max_vote_classes = []
    for num in range(0,y_test.shape[0]):
        max_vote = Counter(predicted_classes[:,num]).most_common(1)
        if max_vote[0][0] == class_col_content[num]:
            accuracy_cnt += 1
        max_vote_classes.append(max_vote[0][0])
    print('Model prediction Accuracy')
    print((accuracy_cnt/y_test.shape[0])*100)

In [32]:
def RandomForest(X_train,Y_train,X_test):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: numpy.ndarray
    """

    X_train = pd.DataFrame(X_train)
    Y_train = pd.DataFrame(Y_train)
    X_test = pd.DataFrame(X_test)
    
    init = datetime.datetime.now()
    n_trees = 10 #number of decision trees in random forest
    tree_predcted_class_list = []
    print('Program running........')   
    for num in range(0,n_trees):
        sample_feature_size = round(math.sqrt(X_train.shape[1]))
        samp_feat_attr=random.sample(range(0,X_train.shape[1]),sample_feature_size)
        print('feature attributes sampled for decision tree %s '%(num+1))
        print(samp_feat_attr)
        clf = DecisionTreeClassifiers(samp_feat_attr,max_depth = 9)
        tree_ = clf.fit(X_train, Y_train)
        predicted_row = clf.predict(X_test)
        tree_predcted_class_list.append(predicted_row)
    final = datetime.datetime.now()
    print("Program starting time")
    print(init)
    print("Program ending time")
    print(final)
    print("Predicted class for 10 different trees")
    predicted_classes = np.array(tree_predcted_class_list)
    print(predicted_classes)

    print('Program terminated........')
    return predicted_classes
    

In [34]:
# Read in data
features = pd.read_csv('Downloads/data.csv')
train_samp_data = features.sample(n = round((features.shape[0]*80)/100), replace = True)
y_train = train_samp_data.iloc[:,-1].to_frame()
x_train = train_samp_data.iloc[:, train_samp_data.columns != y_train.columns.values[0]]

test_sample = features.sample(n = round((features.shape[0]*20)/100))
y_test = test_sample.iloc[:,-1].to_frame()
x_test = test_sample.iloc[:, test_sample.columns != y_test.columns.values[0]]    

predicted_classes = RandomForest(x_train.to_numpy(),y_train.to_numpy(),x_test.to_numpy())
RF_Accuracy(predicted_classes, y_test)

Program running........
feature attributes sampled for decision tree 1 
[39, 29, 26, 44, 36, 19, 18]
feature attributes sampled for decision tree 2 
[33, 27, 1, 21, 14, 40, 11]
feature attributes sampled for decision tree 3 
[14, 44, 7, 24, 30, 39, 35]
feature attributes sampled for decision tree 4 
[29, 44, 18, 7, 42, 32, 41]
feature attributes sampled for decision tree 5 
[40, 23, 18, 2, 17, 41, 7]
feature attributes sampled for decision tree 6 
[7, 41, 13, 36, 3, 32, 35]
feature attributes sampled for decision tree 7 
[12, 2, 46, 17, 41, 39, 24]
feature attributes sampled for decision tree 8 
[0, 46, 47, 19, 35, 17, 34]
feature attributes sampled for decision tree 9 
[10, 14, 4, 15, 0, 36, 43]
feature attributes sampled for decision tree 10 
[20, 34, 28, 31, 16, 9, 47]
Program starting time
2020-03-09 16:22:43.959935
Program ending time
2020-03-09 16:24:03.276261
Predicted class for 10 different trees
[[ 3 10  6 ...  9  4  2]
 [ 3 10  8 ...  9  4  2]
 [ 3  2  4 ...  9  4  2]
 ...
 [